In [33]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta

In [72]:
data_name_list = ["GFSEN","GFSOP","ECMEN","ECMOP"]
data_name = data_name_list[0]

if data_name == "GFSOP":
    window_size = 15
elif data_name == "GFSEN":
    window_size = 15
    sub_delta = -6
elif data_name == "ECMOP":
    window_size = 9
elif data_name == "ECMEN":
    window_size = 14
    sub_delta = -5

In [73]:
if data_name == "GFSOP":
    data = pd.read_csv("./OriginalData/WeatherData/"+data_name+"_WDD_Forecasts_20100101_20210331.csv.gz",sep=',',na_values=['-'], skiprows=[0])
else:
    data = pd.read_csv("./OriginalData/WeatherData/"+data_name+"_WDD_Forecasts_20100101_20210331.csv.gz",sep=',',na_values=['-'])
    

In [74]:
data.head(90)

,REGION,MODEL,INIT_DATE,INIT_HOUR,VERIF_DATE,FLAG,WEIGHT,PARAM,VALUE,10Y_NORMAL,30Y_NORMAL
0,UNITED STATES,gfsen,2010-01-01,0,2010-01-01,F,GAS,HDD,35.62,28.51,30.70
1,UNITED STATES,gfsen,2010-01-01,0,2010-01-02,F,GAS,HDD,39.97,28.67,30.72
2,UNITED STATES,gfsen,2010-01-01,0,2010-01-03,F,GAS,HDD,40.44,28.70,30.80
3,UNITED STATES,gfsen,2010-01-01,0,2010-01-04,F,GAS,HDD,38.50,28.84,30.90
4,UNITED STATES,gfsen,2010-01-01,0,2010-01-05,F,GAS,HDD,37.55,29.04,30.91
...,...,...,...,...,...,...,...,...,...,...,...
85,UNITED STATES,gfsen,2010-01-01,12,2010-01-12,F,POP,CDD,0.00,0.13,0.12
86,UNITED STATES,gfsen,2010-01-01,12,2010-01-13,F,POP,CDD,0.00,0.13,0.11
87,UNITED STATES,gfsen,2010-01-01,12,2010-01-14,F,POP,CDD,0.03,0.13,0.11
88,UNITED STATES,gfsen,2010-01-01,12,2010-01-15,F,POP,CDD,0.00,0.13,0.11


In [75]:
if data_name == "GFSEN":
    data['INIT_HOUR'] = (
        np.select(
            condlist=[data['INIT_HOUR'] == 0, data['INIT_HOUR'] == 6, data['INIT_HOUR'] == 12, data['INIT_HOUR'] == 18], 
            choicelist=["00:00:00", "06:00:00","12:00:00","18:01:00"], 
            default="00:00:00"))
else:
    data['INIT_HOUR'] = (
    np.select(
        condlist=[data['INIT_HOUR'] == 0, data['INIT_HOUR'] == 6, data['INIT_HOUR'] == 12, data['INIT_HOUR'] == 18], 
        choicelist=["00:00:00", "06:00:00","12:00:00","18:00:00"], 
        default="00:00:00"))

In [76]:
data[["10Y_NORMAL","VALUE"]].groupby(data["VALUE"].index // window_size).tail(1)["10Y_NORMAL"].values

array([29.91,  0.13, 29.91, ...,  1.53, 11.54,  1.53])

In [77]:
def mergeData(data,start_date, window_size):
    firstData = data[["INIT_DATE","INIT_HOUR","VERIF_DATE","PARAM","VALUE","10Y_NORMAL"]].groupby(data["VALUE"].index // window_size).head(1)
    firstData["Sum_Sub_Value"] = data[["VALUE"]].groupby(data["VALUE"].index // window_size).apply(lambda x : sum(x["VALUE"].values[sub_delta:])).values
    firstData["Sum_Sub_tail"] = data[["VALUE"]].groupby(data["VALUE"].index // window_size).apply(lambda x : sum(x["VALUE"].values[(sub_delta+1):])).values
    firstData["Sum_Value"]  = data[["VALUE"]].groupby(data["VALUE"].index // window_size).sum().values
    firstData["10Y_NORMAL_LAST"] = data[["10Y_NORMAL"]].groupby(data["10Y_NORMAL"].index // window_size).tail(1).values
    firstData["INIT_Time"] = firstData["INIT_DATE"] + " " + firstData["INIT_HOUR"]
    firstData["SUB_First"] = data[["VALUE"]].groupby(data["VALUE"].index // window_size).apply(lambda x : x["VALUE"].values[sub_delta]).values
    firstData["INIT_Time"] = firstData["INIT_DATE"] + " " + firstData["INIT_HOUR"]
    firstData["INIT_Time"] = pd.to_datetime(firstData["INIT_Time"], format="%Y-%m-%d %H:%M:%S")
    firstData = firstData[(firstData['INIT_Time'] >=  start_date)]
    firstData["VALUE"] = firstData["VALUE"].astype("float64")
    return firstData   

In [78]:
firstData = mergeData(data, start_date='2015-01-01', window_size=window_size)

In [79]:
firstData

,INIT_DATE,INIT_HOUR,VERIF_DATE,PARAM,VALUE,10Y_NORMAL,Sum_Sub_Value,Sum_Sub_tail,Sum_Value,10Y_NORMAL_LAST,INIT_Time,SUB_First
219120,2015-01-01,00:00:00,2015-01-01,HDD,34.39,28.51,190.49,159.00,500.45,29.91,2015-01-01 00:00:00,31.49
219135,2015-01-01,00:00:00,2015-01-01,CDD,0.24,0.19,1.30,1.09,3.70,0.13,2015-01-01 00:00:00,0.21
219150,2015-01-01,06:00:00,2015-01-01,HDD,34.19,28.51,187.07,155.63,493.01,29.91,2015-01-01 06:00:00,31.44
219165,2015-01-01,06:00:00,2015-01-01,CDD,0.24,0.19,1.19,1.04,3.50,0.13,2015-01-01 06:00:00,0.15
219180,2015-01-01,12:00:00,2015-01-02,HDD,31.12,28.67,185.92,155.02,490.88,29.97,2015-01-01 12:00:00,30.90
...,...,...,...,...,...,...,...,...,...,...,...,...
492885,2021-03-31,06:00:00,2021-03-31,CDD,0.96,1.01,5.99,4.26,12.81,1.47,2021-03-31 06:00:00,1.73
492900,2021-03-31,12:00:00,2021-04-01,HDD,20.84,15.06,69.74,59.39,182.02,11.54,2021-03-31 12:00:00,10.35
492915,2021-03-31,12:00:00,2021-04-01,CDD,0.39,1.02,5.37,4.08,12.61,1.53,2021-03-31 12:00:00,1.29
492930,2021-03-31,18:01:00,2021-04-01,HDD,20.91,15.06,69.52,58.33,181.15,11.54,2021-03-31 18:01:00,11.19


In [80]:
def trans_dst(dt, dst_hour, dst_minutes, norm_hour, norm_minutes):
    
    if (((dt >= datetime.strptime("2015-3-8", '%Y-%m-%d'))  & (dt < datetime.strptime("2015-11-1",'%Y-%m-%d'))) |
        ((dt >= datetime.strptime("2016-3-13",'%Y-%m-%d'))  & (dt < datetime.strptime("2016-11-6",'%Y-%m-%d'))) |
        ((dt >= datetime.strptime("2017-3-12",'%Y-%m-%d'))  & (dt < datetime.strptime("2017-11-5",'%Y-%m-%d'))) |
        ((dt >= datetime.strptime("2018-3-11",'%Y-%m-%d'))  & (dt < datetime.strptime("2018-11-4",'%Y-%m-%d'))) |
        ((dt >= datetime.strptime("2019-3-10",'%Y-%m-%d'))  & (dt < datetime.strptime("2019-11-3",'%Y-%m-%d'))) |
        ((dt >= datetime.strptime("2020-3-8",'%Y-%m-%d'))   & (dt < datetime.strptime("2020-11-1",'%Y-%m-%d'))) |
        ((dt >= datetime.strptime("2021-3-14",'%Y-%m-%d'))  & (dt < datetime.strptime("2021-11-7",'%Y-%m-%d')))):
        dt = dt + timedelta(hours=dst_hour,minutes=dst_minutes)
    else:
        dt = dt + timedelta(hours=norm_hour,minutes=norm_minutes)
    return dt

In [81]:
if data_name == "GFSOP":
    firstData["Trans_INIT_Time"] = firstData['INIT_Time'].apply(lambda x:  trans_dst(x, dst_hour=0,dst_minutes=13, norm_hour=0, norm_minutes=-47))    
    firstData['Trans_INIT_Time'] = (
        np.select(
            condlist=[firstData['INIT_HOUR'] == "00:00:00"], 
            choicelist=[firstData['Trans_INIT_Time'] + timedelta(hours=0,minutes=-2)], 
            default=firstData['Trans_INIT_Time']))
elif data_name == "GFSEN":
    firstData["Trans_INIT_Time"] = firstData['INIT_Time'].apply(lambda x:  trans_dst(x, dst_hour=1,dst_minutes=28, norm_hour=0, norm_minutes=28))
elif data_name == "ECMOP":
    firstData["Trans_INIT_Time"] = firstData['INIT_Time'].apply(lambda x:  trans_dst(x, dst_hour=1,dst_minutes=55, norm_hour=0, norm_minutes=55))
elif data_name == "ECMEN":
    firstData["Trans_INIT_Time"] = firstData['INIT_Time'].apply(lambda x:  trans_dst(x, dst_hour=2,dst_minutes=40, norm_hour=1, norm_minutes=40))

In [82]:
firstData.head()

,INIT_DATE,INIT_HOUR,VERIF_DATE,PARAM,VALUE,10Y_NORMAL,Sum_Sub_Value,Sum_Sub_tail,Sum_Value,10Y_NORMAL_LAST,INIT_Time,SUB_First,Trans_INIT_Time
219120,2015-01-01,00:00:00,2015-01-01,HDD,34.39,28.51,190.49,159.00,500.45,29.91,2015-01-01 00:00:00,31.49,2015-01-01 00:28:00
219135,2015-01-01,00:00:00,2015-01-01,CDD,0.24,0.19,1.30,1.09,3.70,0.13,2015-01-01 00:00:00,0.21,2015-01-01 00:28:00
219150,2015-01-01,06:00:00,2015-01-01,HDD,34.19,28.51,187.07,155.63,493.01,29.91,2015-01-01 06:00:00,31.44,2015-01-01 06:28:00
219165,2015-01-01,06:00:00,2015-01-01,CDD,0.24,0.19,1.19,1.04,3.50,0.13,2015-01-01 06:00:00,0.15,2015-01-01 06:28:00
219180,2015-01-01,12:00:00,2015-01-02,HDD,31.12,28.67,185.92,155.02,490.88,29.97,2015-01-01 12:00:00,30.90,2015-01-01 12:28:00


In [83]:
def get_delta(x):
    if x[-1,-1]:
        da = x[-1,0] - (x[0,0])
    else:
        da = x[-1,0] - (x[0,1] + x[-1,2])
    return da

In [84]:
def get_time_flag(x):
    return x.iloc[-1] == x.iloc[0]

In [85]:
def calcuteDelta(df, col_name):
    sub_df= df[df["PARAM"] == col_name].reset_index()
    #sub_df["Time_Flag"] =  sub_df[["Trans_INIT_Time","VERIF_DATE"]].apply(lambda x : 0 if  x["Trans_INIT_Time"].strftime('%Y-%m-%d') == x["VERIF_DATE"] else 1 , axis=1)
    sub_df["Tail_Value"] = sub_df["Sum_Value"] - sub_df["VALUE"]
    sub_df["VERIF_DATE"] = pd.to_datetime(sub_df["VERIF_DATE"])
    sub_df["Time_Flag"] = sub_df["VERIF_DATE"].view(np.int64).rolling(window=2).apply(get_time_flag).astype(bool).values
    sub_df["Delta_Full"] = sub_df[["Sum_Value","Tail_Value","10Y_NORMAL_LAST","Time_Flag"]].rolling(2, method="table", min_periods=0).apply(get_delta, raw=True, engine="numba")["Sum_Value"]
    sub_df["Delta_Sub"]  = sub_df[["Sum_Sub_Value","Sum_Sub_tail","10Y_NORMAL_LAST","Time_Flag"]].rolling(2, method="table", min_periods=0).apply(get_delta, raw=True, engine="numba")["Sum_Sub_Value"]
    return sub_df[['Trans_INIT_Time','VERIF_DATE', 'PARAM', 'VALUE','10Y_NORMAL', 'Sum_Value','Delta_Full',"Delta_Sub"]]

In [86]:
HDD_DATA = calcuteDelta(firstData,"HDD")
CDD_DATA = calcuteDelta(firstData,"CDD")

In [87]:
HDD_DATA

,Trans_INIT_Time,VERIF_DATE,PARAM,VALUE,10Y_NORMAL,Sum_Value,Delta_Full,Delta_Sub
0,2015-01-01 00:28:00,2015-01-01,HDD,34.39,28.51,500.45,0.00,0.00
1,2015-01-01 06:28:00,2015-01-01,HDD,34.19,28.51,493.01,-7.44,-3.42
2,2015-01-01 12:28:00,2015-01-02,HDD,31.12,28.67,490.88,2.09,0.32
3,2015-01-01 18:29:00,2015-01-02,HDD,31.31,28.67,502.42,11.54,13.98
4,2015-01-02 00:28:00,2015-01-02,HDD,31.01,28.67,508.42,6.00,6.12
...,...,...,...,...,...,...,...,...
9123,2021-03-30 19:29:00,2021-03-31,HDD,16.56,15.32,188.50,2.34,1.43
9124,2021-03-31 01:28:00,2021-03-31,HDD,16.18,15.32,187.04,-1.46,-1.40
9125,2021-03-31 07:28:00,2021-03-31,HDD,16.13,15.32,187.57,0.53,3.62
9126,2021-03-31 13:28:00,2021-04-01,HDD,20.84,15.06,182.02,-0.96,-1.31


In [88]:
df_merge = pd.concat([HDD_DATA, CDD_DATA])
df_merge.sort_values(by=['Trans_INIT_Time'],inplace=True)

In [89]:
df_merge.to_csv(data_name+"_WDD_Forecasts_Delta.csv",header=True, index=False, float_format='%.2f')